<a href="https://colab.research.google.com/github/alexjochs/ECE_539_Penguins/blob/model/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
from tensorflow.keras.layers import BatchNormalization
from keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics import mean_squared_error
from keras.initializers import RandomNormal
from keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD
from keras.models import Model,Sequential
from keras.layers import *
from keras import backend as K
from keras.models import model_from_json
from matplotlib import cm as CM
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import scipy.io as io
from PIL import Image
import PIL
import h5py
import os
import glob
import cv2
import random
import math
import sys
import numpy as np

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
K.clear_session()
root = r'/content'
gdrive_data_filepath = os.path.join(root, 'drive', 'MyDrive', 'Colab\ Notebooks', '539\ Project', 'data')
print(root)
print(gdrive_data_filepath)

/content
/content/drive/MyDrive/Colab\ Notebooks/539\ Project/data


In [80]:
def get_new_batch(target=None):
    os.chdir(root)
    assert target is not None, f"can't get specific folder: {target} and load all data"
    tgz_name = target + '.tgz'
    folder_path = os.path.join(gdrive_data_filepath, tgz_name)
    !tar -xf {folder_path} -C {root}
    # cwd = os.path.join(root, target) NECESSARY?? TODO

In [81]:
MASTER_LIST = ['BAILa', 'DAMOa', 'HALFb', 'HALFc', 'LOCKb', 'MAIVb', 'MAIVc', 'NEKOa', 'NEKOb', 'NEKOc', 'PETEc', 'PETEd', 'PETEe', 'PETEf', 'SPIGa', 'GEORa']
# slice working list to split workload between us
working_list = MASTER_LIST[:8] # TODO

In [82]:
# TODO
# for x in working_list:
#     !rm -rf {x}
#     !rm -rf {x}_gt
# !rm -rf /content/content
!rm -rf /content/sample_data/

In [13]:
# Extract all images
for data_split_name in MASTER_LIST:
    if not os.path.exists(data_split_name):
        get_new_batch(data_split_name)
        get_new_batch(data_split_name + '_gt')

tar: /content/drive/MyDrive/Colab Notebooks/539 Project/data/PETEe.tgz: Cannot read: No space left on device
tar: At beginning of tape, quitting now
tar: Error is not recoverable: exiting now
^C
^C
^C
tar: /content/drive/MyDrive/Colab Notebooks/539 Project/data/SPIGa.tgz: Cannot read: No space left on device
tar: At beginning of tape, quitting now
tar: Error is not recoverable: exiting now
tar: /content/drive/MyDrive/Colab Notebooks/539 Project/data/SPIGa_gt.tgz: Cannot read: No space left on device
tar: At beginning of tape, quitting now
tar: Error is not recoverable: exiting now
tar: /content/drive/MyDrive/Colab Notebooks/539 Project/data/GEORa.tgz: Cannot read: No space left on device
tar: At beginning of tape, quitting now
tar: Error is not recoverable: exiting now
tar: /content/drive/MyDrive/Colab Notebooks/539 Project/data/GEORa_gt.tgz: Cannot read: No space left on device
tar: At beginning of tape, quitting now
tar: Error is not recoverable: exiting now


In [83]:
path_sets = [root + '/' + x for x in working_list]
path_sets

['/content/BAILa',
 '/content/DAMOa',
 '/content/HALFb',
 '/content/HALFc',
 '/content/LOCKb',
 '/content/MAIVb',
 '/content/MAIVc',
 '/content/NEKOa']

In [84]:
img_paths = []

for path in path_sets:
    
    for img_path in glob.glob(os.path.join(path, '*.JPG')):
        
        img_paths.append(str(img_path))
        
print("Total images : ", len(img_paths))

Total images :  26242


In [87]:
def create_img(path):
    #Function to load,normalize and return image 
    im = Image.open(path).convert('RGB')
    im = im.resize((800, 600), Image.ANTIALIAS)

    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225

    # print(im.shape)
    #im = np.expand_dims(im,axis  = 0)
    return im

def get_input(path):
    path = path[0] 
    img = create_img(path)
    return(img)
    
    
    
def get_output(path):
    #import target
    #resize target
    
    gt_file = h5py.File(path,'r')
    
    target = np.asarray(gt_file['density'])
    
    img = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64
    
    img = np.expand_dims(img, axis=img.ndim)
    
    # print(img.shape)
    
    return img
    
    
    
def preprocess_input(image,target):
    #crop image
    #crop target
    #resize target
    crop_size = (int(image.shape[0]/2),int(image.shape[1]/2))
    
    
    if random.randint(0,9)<= -1:            
            dx = int(random.randint(0,1)*image.shape[0]*1./2)
            dy = int(random.randint(0,1)*image.shape[1]*1./2)
    else:
            dx = int(random.random()*image.shape[0]*1./2)
            dy = int(random.random()*image.shape[1]*1./2)

    #print(crop_size , dx , dy)
    img = image[dx : crop_size[0]+dx , dy:crop_size[1]+dy]
    
    target_aug = target[dx:crop_size[0]+dx,dy:crop_size[1]+dy]
    #print(img.shape)

    return(img,target_aug)
    
   

In [88]:
#Image data generator 
def image_generator(files, batch_size = 64):
    
    while True:
        
        input_path = np.random.choice(a = files, size = batch_size)
        
        batch_input = []
        batch_output = [] 
          
        #for input_path in batch_paths:
        
        inputt = get_input(input_path )
        out_path = '/content' + input_path[0][:14] + '_gt' + input_path[0][14:].replace('.JPG','_gt.h5')
        if not os.path.exists(out_path):
            continue
        output = get_output(out_path)
            
       
        batch_input += [inputt]
        batch_output += [output]
    

        batch_x = np.array( batch_input )
        batch_y = np.array( batch_output )
        
        yield( batch_x, batch_y )


In [89]:
def save_mod(model , str1 , str2):
    model.save_weights(str1)
    
    model_json = model.to_json()
    
    with open(str2, "w") as json_file:
        json_file.write(model_json)

In [90]:
def init_weights_vgg(model):
    vgg =  VGG16(weights='imagenet', include_top=False)
    
    # json_file = open('models/VGG_16.json', 'r')
    # loaded_model_json = json_file.read()
    # json_file.close()
    # loaded_model = model_from_json(loaded_model_json)
    # loaded_model.load_weights("weights/VGG_16.h5")
    
    # vgg = loaded_model
    
    vgg_weights=[]                         
    for layer in vgg.layers:
        if('conv' in layer.name):
            vgg_weights.append(layer.get_weights())
    
    
    offset=0
    i=0
    while(i<10):
        if('conv' in model.layers[i+offset].name):
            model.layers[i+offset].set_weights(vgg_weights[i])
            i=i+1
            #print('h')
            
        else:
            offset=offset+1

    return (model)
    

In [91]:
def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))


In [92]:
# Neural network model : VGG + Conv
def CrowdNet():  
            #Variable Input Size
            rows = 600
            cols = 800
            
            #Batch Normalisation option
            
            batch_norm = 0
            kernel = (3, 3)
            init = RandomNormal(stddev=0.01)
            model = Sequential() 
            
            #custom VGG:
            
            if(batch_norm):
                model.add(Conv2D(64, kernel_size = kernel, input_shape = (rows,cols,3),activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                
            else:
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same',input_shape = (rows, cols, 3), kernel_initializer = init))
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                
                

                
            #Conv2D
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(1, (1, 1), activation='relu', dilation_rate = 1, kernel_initializer = init, padding = 'same'))
        
            sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
            model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])
            
            model = init_weights_vgg(model)
            
            return model

In [93]:
model = CrowdNet()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [94]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 600, 800, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 600, 800, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 300, 400, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 300, 400, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 300, 400, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 150, 200, 128)    0         
 2D)                                                    

In [95]:
train_gen = image_generator(img_paths)

In [96]:
sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_gen, epochs=15, steps_per_epoch=700 , verbose=1)

(600, 800, 3)
(75, 100, 1)
Epoch 1/15
(600, 800, 3)
(600, 800, 3)
(75, 100, 1)
  1/700 [..............................] - ETA: 4:45:20 - loss: 0.0063 - mse: 0.0022(600, 800, 3)
(75, 100, 1)
  2/700 [..............................] - ETA: 4:17:53 - loss: 0.0032 - mse: 0.0011(600, 800, 3)
(75, 100, 1)
  3/700 [..............................] - ETA: 4:16:29 - loss: nan - mse: nan      (600, 800, 3)
(75, 100, 1)
  4/700 [..............................] - ETA: 4:13:24 - loss: nan - mse: nan(600, 800, 3)
(75, 100, 1)
  5/700 [..............................] - ETA: 4:10:43 - loss: nan - mse: nan(600, 800, 3)
(75, 100, 1)


In [ ]:
save_mod(model,"weights/model_A_weights.h5","models/Model.json")